# Gesture Recognition
#### Submitted by
- Sameer Soin
- Ayush Mandowara

## Problem Statement

A smart tv manufactures wants to add gesture based controls to their TVs. 

To start with, the following 5 gestures are planned to be undstood by the TV:
- Thumbs Up to increase volume
- Thumbs Down to decrease volume
- Left Swipe to move 10 seconds back
- Right Swipe to move 10 seconds ahead
- Open Palm (Stop) to pause

The hardware and software to capture and take action based on the gestures already exists with the manufacturer, our focus will be on `Recognising the Gestures`.

## Data
- The data we have been provided with to train our model consists of images / frames taken in a sequence (videos that are already broken down into images) for various individuals showing the above mentioned hand gestures.  
- The data is labelled with the different classes (gestures) that need to be identified.

## Approach
To do this, we will be using `Deep Learning`. Specifically, we will be trying two approaches:
- Approach 1: 3D CNN Model  
- Approach 2: A CNN + RNN Model


---

# Imports

In [116]:
import cv2
import datetime
import numpy as np
import os
import pandas as pd
import random as rn

from keras import backend as K
import tensorflow as tf
from cv2 import imread
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

### Fixed Random Seeds
- This helps in reproducing results in subsequent runs

In [117]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

## Reading the Data
- The data is labelled
- The file paths along with labels are stored in csv files
- Data is already divided into train and validation folders 

In [118]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

In [119]:
train_doc[0]

'WIN_20180925_18_23_57_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n'

In [120]:
len(train_doc)

663

In [151]:
train_df = pd.read_csv('Project_data/train.csv', delimiter=';', names=['Video Folder', 'Gesture', 'Label'])

In [152]:
len(train_df)

663

In [153]:
train_df.head(3)

,Video Folder,Gesture,Label
0,WIN_20180925_17_08_43_Pro_Left_Swipe_new,Left_Swipe_new,0
1,WIN_20180925_17_18_28_Pro_Left_Swipe_new,Left_Swipe_new,0
2,WIN_20180925_17_18_56_Pro_Left_Swipe_new,Left_Swipe_new,0


In [154]:
train_df.tail(3)

,Video Folder,Gesture,Label
660,WIN_20180907_16_42_05_Pro_Thumbs Up_new,Thumbs Up_new,4
661,WIN_20180907_16_42_55_Pro_Thumbs Up_new,Thumbs Up_new,4
662,WIN_20180907_16_43_39_Pro_Thumbs Up_new,Thumbs Up_new,4


In [155]:
val_df = pd.read_csv('Project_data/val.csv', delimiter=';', names=['Video Folder', 'Gesture', 'Label'])

In [156]:
val_df.head(3)

,Video Folder,Gesture,Label
0,WIN_20180925_17_17_04_Pro_Left_Swipe_new,Left_Swipe_new,0
1,WIN_20180925_17_43_01_Pro_Left_Swipe_new,Left_Swipe_new,0
2,WIN_20180925_18_01_40_Pro_Left_Swipe_new,Left_Swipe_new,0


In [157]:
val_df.tail(3)

,Video Folder,Gesture,Label
97,WIN_20180907_15_54_30_Pro_Thumbs Up_new,Thumbs Up_new,4
98,WIN_20180907_16_10_59_Pro_Thumbs Up_new,Thumbs Up_new,4
99,WIN_20180907_16_39_59_Pro_Thumbs Up_new,Thumbs Up_new,4


In [158]:
train_df = shuffle(train_df, random_state=RANDOM_SEED)

In [159]:
train_df.head(5)

,Video Folder,Gesture,Label
327,WIN_20180925_18_23_57_Pro_Thumbs_Down_new,Thumbs_Down_new,3
579,WIN_20180907_16_21_11_Pro_Stop Gesture_new,Stop Gesture_new,2
513,WIN_20180907_16_38_29_Pro_Left Swipe_new_Left ...,Left Swipe_new_Left Swipe_new,0
362,WIN_20180926_17_23_38_Pro_Thumbs_Down_new,Thumbs_Down_new,3
265,WIN_20180926_17_21_49_Pro_Stop_new,Stop_new,2


In [160]:
val_df = shuffle(val_df, random_state=RANDOM_SEED)

In [161]:
val_df.head(5)

,Video Folder,Gesture,Label
83,WIN_20180907_16_30_54_Pro_Stop Gesture_new,Stop Gesture_new,2
53,WIN_20180925_17_38_43_Pro_Thumbs_Up_new,Thumbs_Up_new,4
70,WIN_20180907_15_55_06_Pro_Right Swipe_new,Right Swipe_new,1
45,WIN_20180926_16_57_50_Pro_Thumbs_Down_new,Thumbs_Down_new,3
44,WIN_20180926_16_44_04_Pro_Thumbs_Down_new,Thumbs_Down_new,3


### Display Some Images

# Generator

### Problems
- Since the data is huge, it cannot be processed in a single go. The machine will throw out of memory error.
- There are images in two types (dimension 120x120 and 360x), we need to make the dimensions same
- There is some room for skipping images to speed up the training process
- Data augmentation may be required to increase accuracy
- Ablation will be required to reduce analysis time

### Solution
All of the above can be achieved with the help of a custom generator which generates data in batches as per requirement.